In [1]:
import time
import copy
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2
%matplotlib qt

from netsurface3d import NetSurf3d
from data4d import Data4d

# import os
# os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

no scikit-tensor detected, so separable functions wont work
This is a python3 issue and can be resolved by:

git clone https://github.com/mnick/scikit-tensor.git
pip3 install scikit-tensor


# Load time-lapse data...

In [2]:
filenames = ['/Users/prakash/Desktop/BobSegDataAndResults/GoodData/2018-06-05_fromMark/20180604_LP823_Control-01_Uncoupled/full_embryo/20180604_LP823_Control-01-01.TIF' ,
             '/Users/prakash/Desktop/BobSegDataAndResults/GoodData/2018-06-05_fromMark/20180604_LP823_Control-01_Uncoupled/full_embryo/20180604_LP823_Control-01-03.TIF']
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t3.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t4.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t5.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t6.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t7.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t8.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t9.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t10.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t11.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t12.TIF']

In [3]:
data = Data4d( filenames, pixelsize=(1.,1.,5.), silent=False )
data.set_seg_params(K=30,max_delta_k=4)
objects = []
ecells = []

Dimensions of frame 0 :  (59, 860, 990)
Dimensions of frame 1 :  (56, 860, 990)


Min radius and max radius at which to look for cells (around center points (seed points))

In [4]:
max_rs = (data.images[0].shape[2] / 8,
          data.images[0].shape[1] / 8,
          data.images[0].shape[0] / 2)

min_rs = (5,5,1)

## adding E cells

In [5]:
Ea = data.init_object("Ea")
objects.append(Ea)
ecells.append(Ea)

Ep = data.init_object("Ep")
objects.append(Ep)
ecells.append(Ep)

In [6]:
#data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), frame_to=11, seed_to=(105,310,16), segment_it=True )
data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(175,415,12), segment_it=True )
data.track(Ea, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ea" in frame 0 with seed coordinates [175. 415.  12.]
      Optimum energy:  80618.0
      Nodes in/out:  9377 5983
Added appearance for "Ea" in frame 1 with seed coordinates [175. 415.  12.]
      Optimum energy:  58065.0
      Nodes in/out:  9871 5489
    Updated center to [176.5625    413.0859375  14.109375 ]


In [7]:
#data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), frame_to=11, seed_to=(140,360,16), segment_it=True )
data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(216,306,12), segment_it=True )
data.track(Ep, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ep" in frame 0 with seed coordinates [216. 306.  12.]
      Optimum energy:  81472.0
      Nodes in/out:  9247 6113
Added appearance for "Ep" in frame 1 with seed coordinates [216. 306.  12.]
      Optimum energy:  56266.0
      Nodes in/out:  9099 6261
    Updated center to [215.546875   304.39257812  13.47460938]


## adding direct neighbors

In [8]:
neighbors = []
for i in range(12):
    n = data.init_object("n"+str(i))
    objects.append(n)
    neighbors.append(n)

In [9]:
nseeds = [( 90,390,12),
          (150,410,12),
          (200,390,12),
          (190,315,12),
          (145,265,12),
          (110,230,12),
          ( 40,255,12),
          ( 45,300,18),
          ( 67,355,18),
          (110,275,24),
          (130,315,24),
          (125,360,24)]
for i in range(len(nseeds)):
    data.add_object_at( neighbors[i], min_rs, max_rs, frame=0, seed=nseeds[i], segment_it=True )
    data.track(neighbors[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "n0" in frame 0 with seed coordinates [ 90. 390.  12.]
      Optimum energy:  76084.0
      Nodes in/out:  5049 10311
Added appearance for "n0" in frame 1 with seed coordinates [ 90. 390.  12.]
      Optimum energy:  55942.0
      Nodes in/out:  3781 11579
    Updated center to [ 85.78515625 393.20507812  12.67773438]
Added appearance for "n1" in frame 0 with seed coordinates [150. 410.  12.]
      Optimum energy:  78748.0
      Nodes in/out:  8930 6430
Added appearance for "n1" in frame 1 with seed coordinates [150. 410.  12.]
      Optimum energy:  55854.0
      Nodes in/out:  9380 5980
    Updated center to [158.37695312 408.76171875  13.9375    ]
Added appearance for "n2" in frame 0 with seed coordinates [200. 390.  12.]
      Optimum energy:  84169.0
      Nodes in/out:  9656 5704
Added appearance for "n2" in frame 1 with seed coordinates [200. 390.  12.]
      Optimum energy:  58044.0
      Nodes in/out:  9245 6115
    Updated center to [193.93945312 396.9492

## adding others

In [10]:
others = []
for i in range(7):
    o = data.init_object("o"+str(i))
    objects.append(o)
    others.append(o)

In [11]:
oseeds = [( 70,205,12),
          (145,170,12),
          ( 50,160,12),
          (105,140,12),
          (190,220,12),
          (200,265,12),
          (220,335,20)]
for i in range(len(oseeds)):
    data.add_object_at( others[i], min_rs, max_rs, frame=0, seed=oseeds[i], segment_it=True )
    data.track(others[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "o0" in frame 0 with seed coordinates [ 70. 205.  12.]
      Optimum energy:  46145.0
      Nodes in/out:  8841 6519
Added appearance for "o0" in frame 1 with seed coordinates [ 70. 205.  12.]
      Optimum energy:  34750.0
      Nodes in/out:  9112 6248
    Updated center to [ 78.140625   212.4375      11.74609375]
Added appearance for "o1" in frame 0 with seed coordinates [145. 170.  12.]
      Optimum energy:  60044.0
      Nodes in/out:  5580 9780
Added appearance for "o1" in frame 1 with seed coordinates [145. 170.  12.]
      Optimum energy:  44051.0
      Nodes in/out:  2351 13009
    Updated center to [145.72460938 169.62304688  11.75976562]
Added appearance for "o2" in frame 0 with seed coordinates [ 50. 160.  12.]
      Optimum energy:  37809.0
      Nodes in/out:  12791 2569
Added appearance for "o2" in frame 1 with seed coordinates [ 50. 160.  12.]
      Optimum energy:  29146.0
      Nodes in/out:  13510 1850
    Updated center to [ 49.6328125  160.972

# Save / Load

In [12]:
data.save('/Users/prakash/Desktop/092216_PH-mCherry-02_w2SD mCherry.pkl')

In [13]:
data = Data4d( [], silent=False )
data.load('/Users/prakash/Desktop/092216_PH-mCherry-02_w2SD mCherry.pkl')
objects = range(len(data.object_names))
ecells = [0,1]
neighbors = range(2,2+12)
others = range(2+12,2+12+7)

Dimensions of frame 0 :  (59, 860, 990)
Dimensions of frame 1 :  (56, 860, 990)
Working on "Ea" (object 1 of 21 )...
   Segmenting in frame 0...


TypeError: object of type 'NoneType' has no len()

# Visualisation stuff 

In [14]:
w = data.show_frame(0,stackUnits=[1.,1.,5.],raise_window=True)
data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
#try:
#    data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
#    data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
#except:
#    None
#data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)

/Users/prakash/anaconda/envs/Py35forBobSeg/lib/python3.5/site-packages/pyopencl/cffi_cl.py:1521: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
for f in range(len(data.images)):
    print ('Exporting frame ',f+1,'of',len(data.images))
    w = data.show_frame(f,stackUnits=[1.,1.,5.],raise_window=False)
    data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
    try:
        data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
        data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
    except:
        None
    data.save_current_visualization( 'frame'+str(f)+'.jpg' )

In [ ]:
data.object_volumes[Ea]